In [1]:

import os


import numpy as np


from matplotlib import pyplot
import pandas as pd


from scipy import optimize

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# tells matplotlib to embed plots within the notebook
%matplotlib inline

In [34]:
df = pd.read_csv("./data/heart.csv")

In [35]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [36]:
print(len(df.columns))

14


In [37]:
a = pd.get_dummies(df['cp'], prefix = "cp")
b = pd.get_dummies(df['thal'], prefix = "thal")
c = pd.get_dummies(df['slope'], prefix = "slope")
df = df.drop(columns = ['cp', 'thal', 'slope'])
frames = [df, a, b, c]
df = pd.concat(frames, axis = 1)

In [38]:
print(df.columns)

Index(['age', 'sex', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang',
       'oldpeak', 'ca', 'target', 'cp_0', 'cp_1', 'cp_2', 'cp_3', 'thal_0',
       'thal_1', 'thal_2', 'thal_3', 'slope_0', 'slope_1', 'slope_2'],
      dtype='object')


In [39]:
df.head()

,age,sex,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,ca,...,cp_1,cp_2,cp_3,thal_0,thal_1,thal_2,thal_3,slope_0,slope_1,slope_2
0,63,1,145,233,1,0,150,0,2.3,0,...,0,0,1,0,1,0,0,1,0,0
1,37,1,130,250,0,1,187,0,3.5,0,...,0,1,0,0,0,1,0,1,0,0
2,41,0,130,204,0,0,172,0,1.4,0,...,1,0,0,0,0,1,0,0,0,1
3,56,1,120,236,0,1,178,0,0.8,0,...,1,0,0,0,0,1,0,0,0,1
4,57,0,120,354,0,1,163,1,0.6,0,...,0,0,0,0,0,1,0,0,0,1


In [40]:
y = df.target.values
x_data = df.drop(['target'], axis = 1)

In [41]:
# Normalize
x = (x_data - np.min(x_data)) / (np.max(x_data) - np.min(x_data)).values

In [42]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=0)

In [46]:
accuracies = {}

lr = LogisticRegression()
lr.fit(x_train,y_train)
acc = lr.score(x_test,y_test)*100

accuracies['Logistic Regression'] = acc
print("Test Accuracy {:.2f}%".format(acc))

Test Accuracy 86.89%


In [47]:
print (len(x_train))

242


In [48]:
print (len(y_train))

242


In [49]:
def sigmoid(z):

    z = np.array(z)
    
    g = np.zeros(z.shape)

    import math
    g = 1 / (1 + math.e**(-z))
    
    return g

## 1 Logistic Regression

In [50]:
# Setup the data matrix appropriately, and add ones for the intercept term
m, n = x_train.shape

# Add intercept term to X
x_train = np.concatenate([np.ones((m, 1)), x_train], axis=1)

In [51]:
def costFunction(theta, X, y):

    # Initialize some useful values
    m = y.size  # number of training examples

    J = 0
    grad = np.zeros(theta.shape)

    
    ytp = np.transpose(y)
    ttp = np.transpose(theta)
    uni = (np.ones(y.shape))
#     print((ytp * np.log(sigmoid(X * theta))))
#     print(np.dot( X, theta))
#     print(y)
    m = np.size(X,0)
    n = np.size(X,1)
#     J = -1/m *  (y * np.log(sigmoid(np.dot( X, theta) + np.subtract(uni, y) * np.log(np.subtract(uni, sigmoid(np.dot( X, theta)))))))
    
    for i in range(m):
        J += -1/m *  (y[i] * np.log(sigmoid(np.dot(ttp,X[i]))) + (1-y[i]) * np.log(1 - sigmoid(np.dot( ttp, X[i]))))
       
    for i in range(n):
        tgrad = 0
        for j in range(m):
            tgrad += ( sigmoid(np.dot(ttp, X[j])) - y[j] ) * X[j,i]
#             print (tgrad)
        grad[i] = 1/m * tgrad

    return J, grad

In [52]:
# # Initialize fitting parameters
# initial_theta = np.zeros(n+1)

# cost, grad = costFunction(initial_theta, x_train, y_train)

# # np.set_printoptions(precision=3)

# print(len(grad))

# print('Cost at initial theta (zeros): {:.3f}'.format(cost))

# print('Gradient at initial theta (zeros):')
# print(grad)


# # Compute and display cost and gradient with non-zero theta
# test_theta = np.ones(n+1) * 0.01
# cost, grad = costFunction(test_theta, x_train, y_train)

# print('Cost at test theta: {:.3f}'.format(cost))

# print('Gradient at test theta:')
# print(grad)


In [53]:
 initial_theta = np.zeros(n+1)
# set options for optimize.minimize

options= {'maxiter': 400}

res = optimize.minimize(costFunction,
                        initial_theta,
                        (x_train, y_train),
                        jac=True,
                        method='TNC',
                        options=options)

# the fun property of `OptimizeResult` object returns
# the value of costFunction at optimized theta
cost = res.fun

# the optimized theta is  in the x property
theta = res.x

# Print theta to screen
print(cost)

print(theta)


0.33397085028108325
[ 1.34925747 -0.01492815 -1.7376987  -1.24288923 -2.44278695 -0.21752259
  0.43747589  2.6508777  -0.758084   -2.99947753 -3.84004045 -0.81230834
  0.05998828  1.0891746   0.9526402  -0.70408209  0.7608791   1.10734506
 -0.23468254  0.82599039  0.0293943   0.52012979]


In [54]:
print(len(theta))

22


In [55]:
m, n = x_test.shape
x_test = np.concatenate([np.ones((m, 1)), x_test], axis=1)

In [56]:
def predict(theta, X):
    
    m = X.shape[0] # Number of training examples


    p = np.zeros(m)


    for i in range (m):
        tmp = sigmoid(np.dot(np.transpose(theta), X[i]))
#         print(tmp)
#         print("-----------" , i)
        if (tmp < 0.5):
            p[i] = 0
        else:
            p[i] = 1

    return p

In [57]:
# Compute accuracy on our training set
p = predict(theta, x_test)
print('Train Accuracy: {:.2f} %'.format(np.mean(p == y_test) * 100))

Train Accuracy: 85.25 %


In [58]:
print (p)
print (len(p))

[0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0.
 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.]
61


In [59]:
print(y_test)
print(len(y_test))

[0 1 0 0 1 0 0 0 0 0 1 1 0 1 1 1 1 1 0 1 1 0 0 0 1 0 0 0 1 1 0 0 1 1 1 0 0
 1 0 0 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 0 1 1 1]
61


In [60]:
def costFunctionReg(theta, X, y, lambda_):
  
    # Initialize some useful values
    m = y.size  # number of training examples

    # You need to return the following variables correctly 
    J = 0
    grad = np.zeros(theta.shape)

    # ===================== YOUR CODE HERE ======================
    
    tmpT = 0
    n = 0
    n = np.size(theta, 0) 
    cost, grad = costFunction(theta, X, y)
    for i in range (n):
        if (i == 0):
            continue
        tmpT += theta[i] * theta[i]
        
        grad[i] = grad[i] + lambda_/m * theta[i]
    J = cost + lambda_ * tmpT /(2*m)
            
    
    # =============================================================
    return J, grad

In [61]:
# Initialize fitting parameters
initial_theta = np.zeros(x_train.shape[1])

# Set regularization parameter lambda to 1 (you should vary this)
lambda_ = 1

# set options for optimize.minimize
options= {'maxiter': 400}

res = optimize.minimize(costFunctionReg,
                        initial_theta,
                        (x_train, y_train, lambda_),
                        jac=True,
                        method='TNC',
                        options=options)

# the fun property of OptimizeResult object returns
# the value of costFunction at optimized theta
cost = res.fun

# the optimized theta is in the x property of the result
theta = res.x


# Compute accuracy on our training set
p = predict(theta, x_test)

print('Train Accuracy: %.2f %%' % (np.mean(p == y_test) * 100))

Train Accuracy: 86.89 %
